In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Kirti
[nltk_data]     Mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+")   ## only word
word=set(stopwords.words("english"))

In [5]:
data_new=pd.read_csv('english_dataset.tsv', sep = '\t', encoding="latin-1")

In [6]:
data_new.shape

(5852, 5)

In [7]:
data_new=data_new.dropna()

In [8]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5852 entries, 0 to 5851
Data columns (total 5 columns):
text_id    5852 non-null object
text       5852 non-null object
task_1     5852 non-null object
task_2     5852 non-null object
task_3     5852 non-null object
dtypes: object(5)
memory usage: 274.3+ KB


In [100]:
data_new = data_new[data_new["task_2"] != "NONE"]


In [101]:
Column_Sequence=np.array(data_new["text"],dtype="str")

In [102]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [103]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [104]:
#load pre-trained embedding:

import io
import os, re, csv, math, codecs
from tqdm import tqdm
##
#####load embeddings
print('loading word embeddings..300 dimention.')
embeddings_index = {}
f = codecs.open('wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

#GLOVE_DIR = "C\:\Users\Abhi\Desktop\Abhinav 2nd imp\Implementation_text and image\2_class"
#embeddings_index = {}
#f = open('glove.6B.300d.txt', encoding="utf8")
#for line in f:
#    values = line.split()
#    word = values[0]
#    coefs = np.asarray(values[1:], dtype='float32')
#    embeddings_index[word] = coefs
#f.close()
#print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

loading word embeddings..300 dimention.


111052it [00:19, 5702.44it/s]


found 111052 word vectors


In [105]:
##### Printing words and index pair 
index=tokenizer.word_index
#print index


###Vocabulary size
vocab_size = len(index)
#print vocab_size

In [106]:

##################################################################################
##################################################################################

#####embedding matrix
print('preparing embedding matrix...')
MAX_NB_WORDS = 800000
embed_dim = 300
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(index))
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

#embedding_matrix = np.random.random((len(tokenizer.word_index) + 1, 300))
#for word, i in tokenizer.word_index.items():
#    embedding_vector = embeddings_index.get(word)
#    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
#        embedding_matrix[i] = embedding_vector
        
#embedding_layer = Embedding(len(tokenizer.word_index) + 1,300,weights=[embedding_matrix],input_length=300,trainable=True)


###############################################################################

preparing embedding matrix...
number of null word embeddings: 1842


In [107]:
max(tokenizer.word_counts.values())

640

In [108]:
len(tokenizer.word_counts)

5301

In [109]:
X_t.shape

(2261, 300)

In [110]:
X_train=X_t

In [111]:
size_embedding = max(list(tokenizer.word_index.values()))

In [112]:
X_train.shape

(2261, 300)

In [113]:
YY=data_new1["task_2"]

In [114]:
len(YY)

2261

In [115]:
#train['sentiment'] = train['sentiment'].map({'OAG': 0, 'CAG': 1,'NAG': 2})
YY = YY.map({'PRFN':0,'OFFN': 1,'HATE': 2 })

In [116]:
#from keras.utils import to_categorical
#labels=data_new['task_2']
#labels= to_categorical(labels,3)
# One-hot encode data
Y_train=np_utils.to_categorical(YY)
#uniqe_class=np.unique(All_class)

In [117]:
Y_train.shape

(2261, 3)

In [118]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.20, random_state=42)

In [119]:
y_train.shape

(1808, 3)

In [120]:
max_len=300

In [121]:
x_train

array([[  97,  417, 2654, ...,    0,    0,    0],
       [4763, 4764, 4765, ...,    0,    0,    0],
       [ 629, 2362, 2363, ...,    0,    0,    0],
       ...,
       [ 718,  193,   55, ...,    0,    0,    0],
       [   3,    5,  502, ...,    0,    0,    0],
       [   9, 1442,  332, ...,    0,    0,    0]])

In [122]:
model = Sequential()
model.add(Embedding(size_embedding+1,100,input_length=max_len))

model.add( Convolution1D(8, 4 ))
model.add( Activation('relu') )

model.add( Convolution1D(16,4) )

model.add( Activation('relu'))
model.add(Convolution1D(64,4))
model.add( MaxPooling1D( pool_size=(4) ) )
model.add(Activation("relu"))
model.add(Convolution1D(64,4))
model.add(Activation("relu"))

model.add( Flatten() )

model.add( Dense(3) )

model.add( Activation('softmax') )

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 100)          530200    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 297, 8)            3208      
_________________________________________________________________
activation_6 (Activation)    (None, 297, 8)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 294, 16)           528       
_________________________________________________________________
activation_7 (Activation)    (None, 294, 16)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 291, 64)           4160      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 72, 64)            0         
__________

In [123]:
from keras.callbacks import ModelCheckpoint
filepath = "task2_cnn_fasttext_300d.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [124]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [125]:
#hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test))
hist=model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test, y_test),callbacks=callbacks_list)

Train on 1808 samples, validate on 453 samples
Epoch 1/100
1808/1808 [==============================] - ETA: 6:08 - loss: 1.0978 - acc: 0.406 - ETA: 3:04 - loss: 1.0990 - acc: 0.328 - ETA: 2:02 - loss: 1.0895 - acc: 0.416 - ETA: 1:31 - loss: 1.0851 - acc: 0.437 - ETA: 1:12 - loss: 1.0867 - acc: 0.431 - ETA: 1:00 - loss: 1.0809 - acc: 0.432 - ETA: 51s - loss: 1.0804 - acc: 0.433 - ETA: 44s - loss: 1.0750 - acc: 0.44 - ETA: 39s - loss: 1.0787 - acc: 0.43 - ETA: 35s - loss: 1.0773 - acc: 0.44 - ETA: 31s - loss: 1.0643 - acc: 0.46 - ETA: 28s - loss: 1.0580 - acc: 0.46 - ETA: 26s - loss: 1.0521 - acc: 0.47 - ETA: 23s - loss: 1.0467 - acc: 0.47 - ETA: 22s - loss: 1.0499 - acc: 0.47 - ETA: 20s - loss: 1.0462 - acc: 0.48 - ETA: 19s - loss: 1.0406 - acc: 0.48 - ETA: 17s - loss: 1.0429 - acc: 0.48 - ETA: 16s - loss: 1.0453 - acc: 0.48 - ETA: 15s - loss: 1.0385 - acc: 0.49 - ETA: 14s - loss: 1.0380 - acc: 0.49 - ETA: 13s - loss: 1.0386 - acc: 0.49 - ETA: 12s - loss: 1.0351 - acc: 0.49 - ETA: 11s 

1808/1808 [==============================] - ETA: 5s - loss: 0.3836 - acc: 0.781 - ETA: 5s - loss: 0.4710 - acc: 0.718 - ETA: 5s - loss: 0.4845 - acc: 0.697 - ETA: 5s - loss: 0.5141 - acc: 0.710 - ETA: 5s - loss: 0.4916 - acc: 0.712 - ETA: 5s - loss: 0.4538 - acc: 0.739 - ETA: 4s - loss: 0.4521 - acc: 0.750 - ETA: 4s - loss: 0.4530 - acc: 0.753 - ETA: 4s - loss: 0.4554 - acc: 0.763 - ETA: 4s - loss: 0.4572 - acc: 0.765 - ETA: 4s - loss: 0.4676 - acc: 0.761 - ETA: 4s - loss: 0.4653 - acc: 0.760 - ETA: 4s - loss: 0.4765 - acc: 0.752 - ETA: 4s - loss: 0.4888 - acc: 0.743 - ETA: 3s - loss: 0.4913 - acc: 0.752 - ETA: 3s - loss: 0.4905 - acc: 0.748 - ETA: 3s - loss: 0.4824 - acc: 0.757 - ETA: 3s - loss: 0.4873 - acc: 0.753 - ETA: 3s - loss: 0.4881 - acc: 0.750 - ETA: 3s - loss: 0.4892 - acc: 0.742 - ETA: 3s - loss: 0.4968 - acc: 0.739 - ETA: 3s - loss: 0.4896 - acc: 0.742 - ETA: 3s - loss: 0.4897 - acc: 0.741 - ETA: 3s - loss: 0.4951 - acc: 0.740 - ETA: 3s - loss: 0.4925 - acc: 0.745 - ETA: 

1808/1808 [==============================] - ETA: 4s - loss: 0.0975 - acc: 0.968 - ETA: 4s - loss: 0.0871 - acc: 0.984 - ETA: 4s - loss: 0.0816 - acc: 0.989 - ETA: 4s - loss: 0.0846 - acc: 0.992 - ETA: 4s - loss: 0.0805 - acc: 0.993 - ETA: 4s - loss: 0.0789 - acc: 0.994 - ETA: 4s - loss: 0.0799 - acc: 0.995 - ETA: 4s - loss: 0.0942 - acc: 0.984 - ETA: 4s - loss: 0.1060 - acc: 0.972 - ETA: 4s - loss: 0.1030 - acc: 0.971 - ETA: 4s - loss: 0.1082 - acc: 0.965 - ETA: 4s - loss: 0.1119 - acc: 0.966 - ETA: 4s - loss: 0.1149 - acc: 0.963 - ETA: 3s - loss: 0.1189 - acc: 0.959 - ETA: 3s - loss: 0.1196 - acc: 0.956 - ETA: 3s - loss: 0.1228 - acc: 0.955 - ETA: 3s - loss: 0.1211 - acc: 0.955 - ETA: 3s - loss: 0.1219 - acc: 0.956 - ETA: 3s - loss: 0.1198 - acc: 0.957 - ETA: 3s - loss: 0.1220 - acc: 0.956 - ETA: 3s - loss: 0.1231 - acc: 0.955 - ETA: 3s - loss: 0.1200 - acc: 0.957 - ETA: 3s - loss: 0.1203 - acc: 0.956 - ETA: 3s - loss: 0.1173 - acc: 0.957 - ETA: 2s - loss: 0.1188 - acc: 0.956 - ETA: 

1808/1808 [==============================] - ETA: 5s - loss: 0.0041 - acc: 1.000 - ETA: 4s - loss: 0.0039 - acc: 1.000 - ETA: 4s - loss: 0.0049 - acc: 1.000 - ETA: 4s - loss: 0.0047 - acc: 1.000 - ETA: 4s - loss: 0.0044 - acc: 1.000 - ETA: 4s - loss: 0.0058 - acc: 1.000 - ETA: 4s - loss: 0.0059 - acc: 1.000 - ETA: 4s - loss: 0.0061 - acc: 1.000 - ETA: 4s - loss: 0.0057 - acc: 1.000 - ETA: 4s - loss: 0.0080 - acc: 1.000 - ETA: 4s - loss: 0.0086 - acc: 1.000 - ETA: 4s - loss: 0.0082 - acc: 1.000 - ETA: 4s - loss: 0.0077 - acc: 1.000 - ETA: 3s - loss: 0.0164 - acc: 0.997 - ETA: 3s - loss: 0.0158 - acc: 0.997 - ETA: 3s - loss: 0.0160 - acc: 0.998 - ETA: 3s - loss: 0.0157 - acc: 0.998 - ETA: 3s - loss: 0.0161 - acc: 0.998 - ETA: 3s - loss: 0.0155 - acc: 0.998 - ETA: 3s - loss: 0.0153 - acc: 0.998 - ETA: 3s - loss: 0.0279 - acc: 0.997 - ETA: 3s - loss: 0.0282 - acc: 0.997 - ETA: 3s - loss: 0.0275 - acc: 0.997 - ETA: 2s - loss: 0.0266 - acc: 0.997 - ETA: 2s - loss: 0.0257 - acc: 0.997 - ETA: 

1808/1808 [==============================] - ETA: 4s - loss: 0.0128 - acc: 1.000 - ETA: 5s - loss: 0.0123 - acc: 1.000 - ETA: 4s - loss: 0.0089 - acc: 1.000 - ETA: 4s - loss: 0.0078 - acc: 1.000 - ETA: 4s - loss: 0.0069 - acc: 1.000 - ETA: 4s - loss: 0.0062 - acc: 1.000 - ETA: 4s - loss: 0.0059 - acc: 1.000 - ETA: 4s - loss: 0.0053 - acc: 1.000 - ETA: 4s - loss: 0.0050 - acc: 1.000 - ETA: 4s - loss: 0.0046 - acc: 1.000 - ETA: 4s - loss: 0.0072 - acc: 0.997 - ETA: 4s - loss: 0.0067 - acc: 0.997 - ETA: 4s - loss: 0.0064 - acc: 0.997 - ETA: 3s - loss: 0.0063 - acc: 0.997 - ETA: 3s - loss: 0.0059 - acc: 0.997 - ETA: 3s - loss: 0.0124 - acc: 0.996 - ETA: 3s - loss: 0.0123 - acc: 0.996 - ETA: 3s - loss: 0.0213 - acc: 0.994 - ETA: 3s - loss: 0.0203 - acc: 0.995 - ETA: 3s - loss: 0.0194 - acc: 0.995 - ETA: 3s - loss: 0.0185 - acc: 0.995 - ETA: 3s - loss: 0.0207 - acc: 0.994 - ETA: 3s - loss: 0.0199 - acc: 0.994 - ETA: 2s - loss: 0.0231 - acc: 0.993 - ETA: 2s - loss: 0.0223 - acc: 0.993 - ETA: 

...

1808/1808 [==============================] - ETA: 3s - loss: 7.9410e-05 - acc: 1.000 - ETA: 3s - loss: 5.8277e-05 - acc: 1.000 - ETA: 3s - loss: 1.0045e-04 - acc: 1.000 - ETA: 3s - loss: 8.2912e-05 - acc: 1.000 - ETA: 3s - loss: 2.0723e-04 - acc: 1.000 - ETA: 3s - loss: 2.0325e-04 - acc: 1.000 - ETA: 3s - loss: 1.8428e-04 - acc: 1.000 - ETA: 3s - loss: 0.0079 - acc: 0.9961    - ETA: 3s - loss: 0.0071 - acc: 0.996 - ETA: 3s - loss: 0.0065 - acc: 0.996 - ETA: 3s - loss: 0.0059 - acc: 0.997 - ETA: 3s - loss: 0.0054 - acc: 0.997 - ETA: 3s - loss: 0.0050 - acc: 0.997 - ETA: 3s - loss: 0.0046 - acc: 0.997 - ETA: 3s - loss: 0.0044 - acc: 0.997 - ETA: 3s - loss: 0.0041 - acc: 0.998 - ETA: 2s - loss: 0.0039 - acc: 0.998 - ETA: 2s - loss: 0.0037 - acc: 0.998 - ETA: 2s - loss: 0.0035 - acc: 0.998 - ETA: 2s - loss: 0.0033 - acc: 0.998 - ETA: 2s - loss: 0.0032 - acc: 0.998 - ETA: 2s - loss: 0.0030 - acc: 0.998 - ETA: 2s - loss: 0.0029 - acc: 0.998 - ETA: 2s - loss: 0.0028 - acc: 0.998 - ETA: 2s - l

1808/1808 [==============================] - ETA: 4s - loss: 2.4641e-04 - acc: 1.000 - ETA: 4s - loss: 1.4754e-04 - acc: 1.000 - ETA: 4s - loss: 2.0136e-04 - acc: 1.000 - ETA: 4s - loss: 0.0068 - acc: 0.9922    - ETA: 4s - loss: 0.0054 - acc: 0.993 - ETA: 4s - loss: 0.0046 - acc: 0.994 - ETA: 3s - loss: 0.0039 - acc: 0.995 - ETA: 3s - loss: 0.0034 - acc: 0.996 - ETA: 3s - loss: 0.0031 - acc: 0.996 - ETA: 3s - loss: 0.0028 - acc: 0.996 - ETA: 3s - loss: 0.0025 - acc: 0.997 - ETA: 3s - loss: 0.0023 - acc: 0.997 - ETA: 3s - loss: 0.0021 - acc: 0.997 - ETA: 3s - loss: 0.0020 - acc: 0.997 - ETA: 3s - loss: 0.0019 - acc: 0.997 - ETA: 3s - loss: 0.0018 - acc: 0.998 - ETA: 3s - loss: 0.0017 - acc: 0.998 - ETA: 3s - loss: 0.0016 - acc: 0.998 - ETA: 3s - loss: 0.0015 - acc: 0.998 - ETA: 3s - loss: 0.0014 - acc: 0.998 - ETA: 2s - loss: 0.0014 - acc: 0.998 - ETA: 2s - loss: 0.0013 - acc: 0.998 - ETA: 2s - loss: 0.0013 - acc: 0.998 - ETA: 2s - loss: 0.0042 - acc: 0.997 - ETA: 2s - loss: 0.0041 - ac

1808/1808 [==============================] - ETA: 4s - loss: 7.8310e-05 - acc: 1.000 - ETA: 4s - loss: 1.7614e-04 - acc: 1.000 - ETA: 4s - loss: 2.8639e-04 - acc: 1.000 - ETA: 4s - loss: 2.4300e-04 - acc: 1.000 - ETA: 4s - loss: 2.5192e-04 - acc: 1.000 - ETA: 4s - loss: 2.1203e-04 - acc: 1.000 - ETA: 3s - loss: 2.1133e-04 - acc: 1.000 - ETA: 3s - loss: 2.1831e-04 - acc: 1.000 - ETA: 3s - loss: 3.8958e-04 - acc: 1.000 - ETA: 3s - loss: 3.5251e-04 - acc: 1.000 - ETA: 3s - loss: 3.3097e-04 - acc: 1.000 - ETA: 3s - loss: 3.1107e-04 - acc: 1.000 - ETA: 3s - loss: 0.0019 - acc: 1.0000    - ETA: 3s - loss: 0.0019 - acc: 1.000 - ETA: 3s - loss: 0.0018 - acc: 1.000 - ETA: 3s - loss: 0.0017 - acc: 1.000 - ETA: 3s - loss: 0.0016 - acc: 1.000 - ETA: 3s - loss: 0.0015 - acc: 1.000 - ETA: 3s - loss: 0.0015 - acc: 1.000 - ETA: 2s - loss: 0.0014 - acc: 1.000 - ETA: 2s - loss: 0.0013 - acc: 1.000 - ETA: 2s - loss: 0.0013 - acc: 1.000 - ETA: 2s - loss: 0.0013 - acc: 1.000 - ETA: 2s - loss: 0.0012 - acc:

1808/1808 [==============================] - ETA: 3s - loss: 0.0011 - acc: 1.000 - ETA: 3s - loss: 6.3495e-04 - acc: 1.000 - ETA: 4s - loss: 4.3917e-04 - acc: 1.000 - ETA: 4s - loss: 0.0058 - acc: 1.0000    - ETA: 3s - loss: 0.0046 - acc: 1.000 - ETA: 3s - loss: 0.0039 - acc: 1.000 - ETA: 3s - loss: 0.0035 - acc: 1.000 - ETA: 3s - loss: 0.0030 - acc: 1.000 - ETA: 3s - loss: 0.0032 - acc: 1.000 - ETA: 3s - loss: 0.0029 - acc: 1.000 - ETA: 3s - loss: 0.0027 - acc: 1.000 - ETA: 3s - loss: 0.0025 - acc: 1.000 - ETA: 3s - loss: 0.0023 - acc: 1.000 - ETA: 3s - loss: 0.0022 - acc: 1.000 - ETA: 3s - loss: 0.0021 - acc: 1.000 - ETA: 3s - loss: 0.0020 - acc: 1.000 - ETA: 3s - loss: 0.0019 - acc: 1.000 - ETA: 3s - loss: 0.0018 - acc: 1.000 - ETA: 3s - loss: 0.0026 - acc: 1.000 - ETA: 2s - loss: 0.0025 - acc: 1.000 - ETA: 2s - loss: 0.0024 - acc: 1.000 - ETA: 2s - loss: 0.0023 - acc: 1.000 - ETA: 2s - loss: 0.0023 - acc: 1.000 - ETA: 2s - loss: 0.0022 - acc: 1.000 - ETA: 2s - loss: 0.0021 - acc: 1

1808/1808 [==============================] - ETA: 4s - loss: 2.1073e-04 - acc: 1.000 - ETA: 3s - loss: 1.7699e-04 - acc: 1.000 - ETA: 3s - loss: 7.6312e-04 - acc: 1.000 - ETA: 3s - loss: 0.0028 - acc: 1.0000    - ETA: 3s - loss: 0.0035 - acc: 1.000 - ETA: 3s - loss: 0.0087 - acc: 1.000 - ETA: 3s - loss: 0.0483 - acc: 0.982 - ETA: 3s - loss: 0.0433 - acc: 0.984 - ETA: 3s - loss: 0.0455 - acc: 0.982 - ETA: 3s - loss: 0.0410 - acc: 0.984 - ETA: 3s - loss: 0.0373 - acc: 0.985 - ETA: 3s - loss: 0.0352 - acc: 0.987 - ETA: 3s - loss: 0.0477 - acc: 0.985 - ETA: 3s - loss: 0.0450 - acc: 0.986 - ETA: 3s - loss: 0.0444 - acc: 0.985 - ETA: 3s - loss: 0.0610 - acc: 0.982 - ETA: 3s - loss: 0.0634 - acc: 0.979 - ETA: 3s - loss: 0.0599 - acc: 0.980 - ETA: 3s - loss: 0.0568 - acc: 0.981 - ETA: 2s - loss: 0.0552 - acc: 0.981 - ETA: 2s - loss: 0.0527 - acc: 0.982 - ETA: 2s - loss: 0.0504 - acc: 0.983 - ETA: 2s - loss: 0.0483 - acc: 0.983 - ETA: 2s - loss: 0.0464 - acc: 0.984 - ETA: 2s - loss: 0.0524 - ac

In [52]:
#model.save("task2_cnn_glove6B_300d.hdf5")

In [126]:
from keras.models import load_model
# load weights into new model
loaded_model= load_model("task2_cnn_fasttext_300d.hdf5")

In [68]:
#loaded_model.evaluate(x_test,y_test)

1171/1171 [==============================] - 2s 2ms/step


[0.44159719808604553, 0.8134073443539014]

In [131]:
w=model.predict_classes(x_test)

In [132]:
from sklearn.metrics import classification_report,confusion_matrix

In [133]:
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [134]:
print(classification_report(Y_actual,w))

             precision    recall  f1-score   support

          0       0.39      0.42      0.40       146
          1       0.18      0.39      0.25        76
          2       0.60      0.35      0.44       231

avg / total       0.46      0.38      0.40       453



In [135]:
print(confusion_matrix(Y_actual, w))

[[61 54 31]
 [24 30 22]
 [72 79 80]]


In [136]:
data_test=pd.read_csv('hasoc2019_en_test.tsv', sep = '\t', encoding="latin-1")

In [137]:
data_test.head()

,text_id,text
0,hasoc_en_902,West Bengal Doctor Crisis: Protesting doctors ...
1,hasoc_en_416,68.5 million people have been forced to leave ...
2,hasoc_en_207,"You came, you saw .... we will look after the ..."
3,hasoc_en_595,We'll get Brexit delivered by October 31st. ...
4,hasoc_en_568,Fuck you. Go back to the dark ages you cow @IB...


In [138]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen)
    wordss=[(Word) for Word in sen if len(Word)>2 and Word not in word]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [139]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen padding 100

In [140]:
w_test=model.predict_classes(X_t)

In [141]:
w_test

array([0, 0, 0, ..., 0, 2, 1], dtype=int64)

In [142]:
print(len(w_test)-np.count_nonzero(w_test))
np.count_nonzero(w_test)

313


840

In [143]:
output = pd.DataFrame()
#out_result['text_id'] = 

In [144]:
output['text_id'] = data_test['text_id']

In [145]:
output.tail()

,text_id
1148,hasoc_en1_3958
1149,hasoc_en1_4648
1150,hasoc_en1_4832
1151,hasoc_en1_3721
1152,hasoc_en1_991


In [146]:
ans_dict = {0:'PRFN',1:'OFFN',2:'HATE'}
w2 = np.vectorize(ans_dict.get)(w_test)
output['result'] = w2

In [147]:
output.to_csv('Kirti Kumari_English_task_2_run_3.tsv',header=True,sep='\t',index=False)